In [38]:
import numpy as np
import pandas as pd
import re
from transformers import BertModel, AutoTokenizer
import torch
import warnings

In [39]:
warnings.filterwarnings("ignore")

In [4]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = BertModel.from_pretrained("cointegrated/rubert-tiny2")

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

In [28]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [29]:
texts = ["Измерение косинусного подобия", 
         "Кластеризация на основе значений косинусного расстояния", 
         "K-среднее значение с косинусным расстоянием", 
         "В Лондоне передумали объявлять России новую холодную войну", 
         "Борис Джонсон: Запад не находится в состоянии новой холодной войны с РФ",
         "В британском МИД заявили, что Запад не хочет новой холодной войны с Россией"
        ]

In [60]:
def norm_text(text):
    new_text = "".join([x for x in text.lower() if x.isalpha() or x==" "])
    return new_text

In [63]:
texts_norm = list(map(norm_text, texts))

In [76]:
texts_lower = list(map(lambda x: x.lower(), texts))

In [30]:
embeddings = [embed_bert_cls(txt, model, tokenizer) for txt in texts]

In [65]:
norm_emb = [embed_bert_cls(txt, model, tokenizer) for txt in texts_norm]

In [77]:
lower_emb = [embed_bert_cls(txt, model, tokenizer) for txt in texts_lower]

In [31]:
def cos_sim(x,y):
    delta = x - y
    cos_sim = np.dot(x,y) / (np.linalg.norm(x) * np.linalg.norm(y))
    return cos_sim

In [78]:
for emb in lower_emb[:]:
    print(cos_sim(lower_emb[-1], emb))

0.43541726
0.40410042
0.47023407
0.7917399
0.7915406
1.0000001


In [79]:
from sklearn.cluster import KMeans

In [88]:
inertia, sil = [], []
for i in range(2,5):
    tmp_kmeans = KMeans(n_clusters=i)
    tmp_kmeans.fit(lower_emb)
    tmp_labels = tmp_kmeans.predict(lower_emb)
    sil.append(silhouette_score(lower_emb, tmp_labels, metric="cosine"))
    inertia.append(tmp_kmeans.inertia_)

In [89]:
inertia

[0.9022842597957828, 0.6243528187591632, 0.4023459141492939]

In [90]:
sil

[0.5837504, 0.3468697, 0.08748483]

In [70]:
from sklearn.metrics import silhouette_score

In [74]:
kmeans = KMeans(n_clusters=3)
y_labels = kmeans.fit_predict(embeddings)

In [75]:
y_labels

array([1, 1, 1, 0, 2, 2], dtype=int32)

In [73]:
silhouette_score(embeddings, y_labels, metric="cosine")

0.6301982